In [1]:
!pip install mediapipe opencv-python


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a) # First point
    b = np.array(b) # Mid point
    c = np.array(c) # End point
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

# Function to create rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, radius, color, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw the filled rectangle without corners
    cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y1 + radius), color, thickness)  # Top
    cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)  # Middle
    cv2.rectangle(img, (x1 + radius, y2 - radius), (x2 - radius, y2), color, thickness)  # Bottom
    
    # Draw the four corner circles
    cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)  # Top-left
    cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)  # Top-right
    cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)  # Bottom-left
    cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)  # Bottom-right

cap = cv2.VideoCapture(0)

# Neck rotation counter variables
counter = 0 
stage = None
rotation_angle = 0  # Track rotation angle
feedback = ""
max_right_rotation = 0  # Track maximum right rotation
max_left_rotation = 0  # Track maximum left rotation
left_done = False  # Track if left rotation is completed in the current rep
right_done = False  # Track if right rotation is completed in the current rep

# UI parameters
panel_width = 400
panel_height = 100
instruction_panel_height = 160  # Increased height for instructions
corner_radius = 20
bg_color = (30, 30, 30)  # Dark background for minimalist look
text_color = (255, 255, 255)  # White text
accent_color = (0, 200, 255)  # Cyan accent
progress_color = (0, 255, 200)  # Teal for progress

# Debug info
debug_mode = False  # Set to True to show angle values

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for neck rotation detection
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate the midpoint between shoulders (reference for center position)
            mid_shoulders = [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2]
            
            # Calculate horizontal position of nose relative to mid-shoulders
            # Positive value means head is turned right, negative means left
            relative_nose_pos = nose[0] - mid_shoulders[0]
            
            # Calculate the distance between ears (to normalize rotation measure)
            ear_distance = np.linalg.norm(np.array(left_ear) - np.array(right_ear))
            
            # Normalize the rotation value to percentage (-100% to +100%)
            # where -100% is full left rotation, 0% is center, and +100% is full right rotation
            rotation_angle = (relative_nose_pos / (ear_distance * 0.5)) * 100
            
            # Limit to -100 to 100 range
            rotation_angle = max(-100, min(100, rotation_angle))
            
            # Debug info
            if debug_mode:
                cv2.putText(image, f"Rotation: {int(rotation_angle)}%", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f"Stage: {stage}", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f"L:{left_done} R:{right_done}", (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # NECK ROTATION COUNTER LOGIC
            # Threshold for considering a rotation complete (adjust sensitivity as needed)
            rotation_threshold = 40  # Percentage of full rotation
            
            # Center/neutral position
            if abs(rotation_angle) < 20:
                if stage == 'rotating':
                    stage = "center"
                    # If both left and right rotations were completed, count a rep
                    if left_done and right_done:
                        counter += 1
                        feedback = "Rep counted!"
                        left_done = False
                        right_done = False
                elif stage is None:
                    stage = "center"
                    feedback = "Ready"
            
            # Rotating left or right
            else:
                if stage == 'center' or stage is None:
                    stage = "rotating"
                
                # Track left rotation
                if rotation_angle <= -rotation_threshold:
                    left_done = True
                    feedback = f"Left rotation: {abs(int(rotation_angle))}%"
                
                # Track right rotation
                elif rotation_angle >= rotation_threshold:
                    right_done = True
                    feedback = f"Right rotation: {int(rotation_angle)}%"
                
                # Update max rotations
                max_left_rotation = min(max_left_rotation, rotation_angle)
                max_right_rotation = max(max_right_rotation, rotation_angle)
                    
        except:
            pass
        
        # Get frame dimensions
        h, w, _ = image.shape
        
        # Create minimalist UI panels
        bottom_panel_x = (w - panel_width) // 2  # Center horizontally
        bottom_panel_y = h - panel_height - 20   # Bottom with margin
        
        top_panel_x = (w - panel_width) // 2     # Center horizontally
        top_panel_y = 20                         # Top with margin
        
        # Draw semi-transparent background for instruction panel (top)
        instruction_overlay = image.copy()
        draw_rounded_rectangle(
            instruction_overlay, 
            (top_panel_x, top_panel_y), 
            (top_panel_x + panel_width, top_panel_y + instruction_panel_height), 
            corner_radius, 
            bg_color, 
            -1
        )
        
        # Apply instruction overlay with transparency
        alpha = 0.8
        cv2.addWeighted(instruction_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Add instruction title
        cv2.putText(image, "NECK ROTATION EXERCISE", 
                   (top_panel_x + panel_width//2 - 120, top_panel_y + 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, accent_color, 1, cv2.LINE_AA)
        
        # Add step instructions - improved spacing and positioning
        instructions = [
            "1. Start in center position facing camera",
            "2. Rotate head to left until detected",
            "3. Return to center",
            "4. Rotate head to right until detected",
            "5. Return to center to complete one rep"
        ]
        
        # More space between lines and smaller font
        line_spacing = 22  # Increased from 20
        start_y = top_panel_y + 60  # Moved down slightly
        
        for i, instr in enumerate(instructions):
            y_pos = start_y + (i * line_spacing)
            cv2.putText(image, instr, 
                       (top_panel_x + 25, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, text_color, 1, cv2.LINE_AA)
        
        # Draw semi-transparent background for counter panel (bottom)
        counter_overlay = image.copy()
        draw_rounded_rectangle(
            counter_overlay, 
            (bottom_panel_x, bottom_panel_y), 
            (bottom_panel_x + panel_width, bottom_panel_y + panel_height), 
            corner_radius, 
            bg_color, 
            -1
        )
        
        # Apply counter overlay with transparency
        alpha = 0.8
        cv2.addWeighted(counter_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Draw rotation gauge - horizontal bar with center indicator
        gauge_height = 6
        gauge_y = bottom_panel_y + panel_height - 20
        gauge_width = panel_width - 60  # Slightly narrower
        
        # Background of gauge
        cv2.rectangle(image, 
                     (bottom_panel_x + 30, gauge_y), 
                     (bottom_panel_x + 30 + gauge_width, gauge_y + gauge_height), 
                     (80, 80, 80), -1, cv2.LINE_AA)
        
        # Center marker
        center_x = bottom_panel_x + 30 + gauge_width // 2
        cv2.rectangle(image, 
                     (center_x - 1, gauge_y - 2), 
                     (center_x + 1, gauge_y + gauge_height + 2), 
                     (150, 150, 150), -1, cv2.LINE_AA)
        
        # Fill based on current rotation (-100 to +100)
        fill_start = center_x
        fill_width = int((rotation_angle / 100) * (gauge_width / 2))
        
        if rotation_angle < 0:  # Left rotation (negative value)
            fill_start = center_x + fill_width
            fill_width = abs(fill_width)
        
        cv2.rectangle(image, 
                     (fill_start, gauge_y), 
                     (fill_start + fill_width, gauge_y + gauge_height), 
                     progress_color, -1, cv2.LINE_AA)
        
        # Add gauge labels
        cv2.putText(image, "L", 
                   (bottom_panel_x + 30, gauge_y + gauge_height + 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        
        cv2.putText(image, "R", 
                   (bottom_panel_x + 30 + gauge_width - 10, gauge_y + gauge_height + 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        
        # Add "NECK ROTATIONS" label
        cv2.putText(image, "NECK ROTATIONS", 
                   (bottom_panel_x + panel_width//2 - 72, bottom_panel_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1, cv2.LINE_AA)
        
        # Counter in smaller font and placed below the label
        cv2.putText(image, str(counter), 
                   (bottom_panel_x + panel_width//2 - (15 if counter < 10 else 25), bottom_panel_y + 55), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, text_color, 2, cv2.LINE_AA)
        
        # Current state and feedback
        if feedback:
            cv2.putText(image, feedback, 
                       (bottom_panel_x + 20, bottom_panel_y + 85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, accent_color, 1, cv2.LINE_AA)
        
        # Visualization of completed sides in current rep
        if left_done:
            cv2.circle(image, (bottom_panel_x + 30, bottom_panel_y + 55), 5, progress_color, -1)
        if right_done:
            cv2.circle(image, (bottom_panel_x + panel_width - 30, bottom_panel_y + 55), 5, progress_color, -1)
        
        # Draw key head and shoulder landmarks for visualization with 80% transparency
        if results.pose_landmarks:
            # Create overlay for landmarks
            landmarks_overlay = image.copy()
            
            # Key points to show
            key_landmarks = [
                mp_pose.PoseLandmark.NOSE,
                mp_pose.PoseLandmark.LEFT_EAR,
                mp_pose.PoseLandmark.RIGHT_EAR,
                mp_pose.PoseLandmark.LEFT_EYE,
                mp_pose.PoseLandmark.RIGHT_EYE,
                mp_pose.PoseLandmark.LEFT_SHOULDER,
                mp_pose.PoseLandmark.RIGHT_SHOULDER
            ]
            
            # Draw key landmarks on overlay
            for landmark_id in key_landmarks:
                landmark = results.pose_landmarks.landmark[landmark_id.value]
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                # Draw larger circles for key points
                cv2.circle(landmarks_overlay, (cx, cy), 5, accent_color, -1)
            
            # Draw connections between landmarks on overlay
            connections = [
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_EAR),
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_EAR),
                (mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.NOSE),
                (mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.NOSE),
                (mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.NOSE),
                (mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.NOSE),
                (mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EAR),
                (mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EAR),
            ]
            
            for connection in connections:
                start_point = results.pose_landmarks.landmark[connection[0].value]
                end_point = results.pose_landmarks.landmark[connection[1].value]
                
                start_x, start_y = int(start_point.x * w), int(start_point.y * h)
                end_x, end_y = int(end_point.x * w), int(end_point.y * h)
                
                cv2.line(landmarks_overlay, (start_x, start_y), (end_x, end_y), progress_color, 2)
            
            # Apply landmarks with 80% transparency
            landmarks_alpha = 0.2  # 20% opacity (80% transparency)
            cv2.addWeighted(landmarks_overlay, landmarks_alpha, image, 1 - landmarks_alpha, 0, image)
        
        cv2.imshow('Neck Rotation Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()